In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, cohen_kappa_score, balanced_accuracy_score
from itertools import combinations

import warnings
warnings.filterwarnings("ignore")

In [10]:
# read the pre-processed data all agreed
google_ngrams_data = pd.read_csv('../Google_Ngrams/features_google_ngram.csv', index_col=None)
bnc_data = pd.read_csv('../corpora_size/size_differential_features.csv', index_col=None)['bnc_100m_sum']/10000000000
base_feature = ['nrdirhypers_x',
                'nrhypos_x',
                'nrpartrels_normalised_x',
                'depthfromtopsynset_normalised_x',
                'glosslength_normalised_x',
                'minwordlength_x',
                'nroflemmas_x',
                'polyscore_max_x']

target = ['vote_x']

data = pd.concat([google_ngrams_data, bnc_data], axis=1)

data

,Synsets,domain_x,nrdirhypers_x,nrhypos_x,nrpartrels_normalised_x,depthfromtopsynset_normalised_x,glosslength_normalised_x,minwordlength_x,nroflemmas_x,polyscore_max_x,...,ngram_50y_max,ngram_100y_mean,ngram_100y_max,ngram_200y_mean,ngram_200y_max,ngram_400y_mean,ngram_400y_max,ngram_500y_mean,ngram_500y_max,bnc_100m_sum
0,Synset('adjustable_wrench.n.01'),tool,1,7,0.0,1.012903,0.563173,17,2,1,...,5.067315e-08,2.252890e-08,5.067315e-08,1.380492e-08,5.067315e-08,6.919673e-09,5.067315e-08,5.336133e-09,5.067315e-08,0.000000e+00
1,Synset('allen_wrench.n.01'),tool,1,0,0.0,1.012903,0.391092,12,1,1,...,6.483798e-08,1.515313e-08,6.483798e-08,7.715051e-09,6.483798e-08,3.867145e-09,6.483798e-08,2.982164e-09,6.483798e-08,0.000000e+00
2,Synset('alligator_wrench.n.01'),tool,1,0,0.0,1.012903,1.517437,16,1,1,...,5.970550e-10,6.184165e-10,7.804026e-09,1.237592e-09,1.499339e-08,6.203393e-10,1.499339e-08,4.783770e-10,1.499339e-08,0.000000e+00
3,Synset('awl.n.01'),tool,1,2,15.7,0.911613,0.985552,3,1,1,...,3.752697e-07,3.562540e-07,6.002532e-07,5.869817e-07,2.510665e-06,6.210211e-07,6.602901e-06,5.069033e-07,6.602901e-06,3.900000e-09
4,Synset('backsaw.n.01'),tool,1,0,0.0,1.114194,1.110701,7,2,1,...,2.446695e-08,1.704462e-08,2.446695e-08,1.451245e-08,3.507272e-08,9.018908e-09,2.260505e-07,6.954965e-09,2.260505e-07,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,Synset('ballet_skirt.n.01'),garm,1,0,0.0,0.947552,0.578283,4,2,2,...,8.004256e-07,2.485961e-07,8.004256e-07,1.531570e-07,8.004256e-07,1.136149e-07,8.004256e-07,9.223624e-08,1.686392e-06,3.300000e-09
835,Synset('mess_jacket.n.01'),garm,1,0,0.0,1.158120,1.652238,11,3,1,...,8.272560e-09,1.233276e-08,2.136213e-08,1.358353e-08,5.083992e-08,6.963715e-09,5.547362e-08,5.370096e-09,5.547362e-08,0.000000e+00
836,Synset('long_johns.n.01'),garm,1,0,0.0,1.052836,0.479149,10,1,1,...,1.103470e-07,2.680743e-08,1.103470e-07,1.393969e-08,1.103470e-07,6.987224e-09,1.103470e-07,5.388225e-09,1.103470e-07,0.000000e+00
837,Synset('undies.n.01'),garm,1,0,0.0,1.158120,0.280880,6,1,1,...,2.565946e-07,5.787213e-08,2.565946e-07,3.095668e-08,2.565946e-07,2.025520e-08,3.896998e-07,1.561987e-08,3.896998e-07,3.300000e-09


## GlobalModel Test

In [4]:
# split training set and testing set using K-Flod
def new_features_global_model_test(dataset, feature, new_features, target):
    K = 10
    random_seed = 7 # R
    data = dataset.reset_index()
    if new_features is None:
        feature_list = feature
    else:
        feature_list = feature + new_features
    X = data[feature_list]
    y = data[target]

    K_Flod = StratifiedKFold(n_splits=K, shuffle=True, random_state=random_seed)
    K_Flod.get_n_splits(X, y)
    cohen_kappa = []
    balanced_acc = []
    for train_index, test_index in K_Flod.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # SMOTE algorithm
        smote = SMOTE(random_state=random_seed, k_neighbors=2)
        X_train, y_train = smote.fit_resample(X_train, y_train)

        # define random forest model
        rf = RandomForestClassifier(random_state=random_seed, max_features='sqrt', n_estimators=1400, min_samples_split=2, min_samples_leaf=1, max_depth=50, oob_score=True, criterion='gini', bootstrap=True).fit(X_train, y_train)

        # predict and make score
        pipeline = make_pipeline(smote, rf)
        y_pred = pipeline.predict(X_test)

        kappa = cohen_kappa_score(y_test, y_pred)
        cohen_kappa.append(kappa)
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        balanced_acc.append(balanced_accuracy)

    result_kappa = np.mean(cohen_kappa)
    results_acc = np.mean(balanced_acc)

    return result_kappa, results_acc

In [16]:
new_features_global_model_test(data, base_feature, ['ngram_50y_max', 'ngram_100y_max', 'ngram_400y_mean', 'ngram_500y_max'], target)

(0.7144327457398147, 0.8646313885119856)

## LocalModel Test

In [7]:
# generate the local dataframe for different domains
local_fruit = data.loc[data['domain_x']=='fruit']
local_tool = data.loc[data['domain_x']=='tool']
local_music = data.loc[data['domain_x']=='music']
local_furniture = data.loc[data['domain_x']=='furn']
local_garments = data.loc[data['domain_x']=='garm']

local_list = [local_fruit, local_tool, local_music, local_furniture, local_garments]

def new_feature_local_model_test(dataset_list, base_feature, new_feature, target):
    random_seed = 7 # R
    K = 10
    if new_feature is None:
        feature_list = base_feature
    else:
        feature_list = base_feature + new_feature

    local_kappa_list = []
    local_balancedAcc_list = []
    for dataset in dataset_list:
        X = dataset[feature_list]
        y = dataset[target]
        K_Flod = StratifiedKFold(n_splits=K, shuffle=True, random_state=random_seed)
        K_Flod.get_n_splits(X, y)
        cohen_kappa = []
        balanced_acc = []
        for train_index, test_index in K_Flod.split(X, y):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # SMOTE algorithm
            smote = SMOTE(random_state=random_seed, k_neighbors=2)
            X_train, y_train = smote.fit_resample(X_train, y_train)

            # define random forest model
            rf = RandomForestClassifier(random_state=random_seed, max_features='sqrt', n_estimators=1400, min_samples_split=2, min_samples_leaf=1, max_depth=50, oob_score=True, criterion='gini', bootstrap=True).fit(X_train, y_train)

            # predict and make score
            pipeline = make_pipeline(smote, rf)
            y_pred = pipeline.predict(X_test)

            kappa = cohen_kappa_score(y_test, y_pred)
            cohen_kappa.append(kappa)
            balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
            balanced_acc.append(balanced_accuracy)

        cohen_kappa_mean = np.mean(cohen_kappa)
        balanced_acc_mean = np.mean(balanced_acc)
        local_kappa_list.append(cohen_kappa_mean)
        local_balancedAcc_list.append(balanced_acc_mean)

    local_kappa = np.mean(local_kappa_list)
    local_acc = np.mean(local_balancedAcc_list)

    return local_kappa, local_acc

In [11]:
new_feature_local_model_test(local_list, base_feature, ['ngram_100y_max', 'ngram_400y_mean'], target)

(0.7115015229441634, 0.8622309651133181)

## TransferModel Test

In [21]:
# generate the local dataframe for different domains
local_fruit = data.loc[data['domain_x']=='fruit']
local_tool = data.loc[data['domain_x']=='tool']
local_music = data.loc[data['domain_x']=='music']
local_furniture = data.loc[data['domain_x']=='furn']
local_garments = data.loc[data['domain_x']=='garm']

local_list = [local_fruit, local_tool, local_music, local_furniture, local_garments]

# generate the unseen(transfer) dataframe for different domains
unseen_fruit = data.loc[data['domain_x']!='fruit']
unseen_tool = data.loc[data['domain_x']!='tool']
unseen_music = data.loc[data['domain_x']!='music']
unseen_furniture = data.loc[data['domain_x']!='furn']
unseen_garments = data.loc[data['domain_x']!='garm']

unseen_list = [unseen_fruit, unseen_tool, unseen_music, unseen_furniture, unseen_garments]

def new_feature_transfer_model_test(train_list, test_list, base_feature, new_feature, target):
    random_seed = 7 # R
    transfer_kappa_list = []
    transfer_balancedAcc_list = []
    for counter in range(len(train_list)):
        training_data = train_list[counter].reset_index()
        testing_data = test_list[counter].reset_index()
        if new_feature is None:
            feature_list = base_feature
        else:
            feature_list = base_feature + new_feature
        X_train = training_data[feature_list]
        y_train = training_data[target]
        X_test = testing_data[feature_list]
        y_test = testing_data[target]

        # SMOTE algorithm
        smote = SMOTE(random_state=random_seed, k_neighbors=2)
        X_train, y_train = smote.fit_resample(X_train, y_train)

        # define random forest model
        rf = RandomForestClassifier(random_state=random_seed, max_features='sqrt', n_estimators=1400, min_samples_split=2, min_samples_leaf=1, max_depth=50, oob_score=True, criterion='gini', bootstrap=True).fit(X_train, y_train)

        # predict and make score
        pipeline = make_pipeline(smote, rf)
        y_pred = pipeline.predict(X_test)

        kappa = cohen_kappa_score(y_test, y_pred)
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)

        transfer_kappa_list.append(kappa)
        transfer_balancedAcc_list.append(balanced_accuracy)

    transfer_kappa = np.mean(transfer_kappa_list), np.mean(transfer_balancedAcc_list)


    return transfer_kappa

In [24]:
new_feature_transfer_model_test(unseen_list, local_list, base_feature, ['ngram_5y_mean', 'ngram_50y_mean', 'ngram_100y_mean', 'ngram_200y_max', 'ngram_500y_max'], target)

(0.5779510418076527, 0.824327867292346)